# Test PHYSLITE Combination

This notebook will test ideas for combining the calibrated values from PHYSLITE and the uncalibrated values from LLP1.

* Should be faster than running calibrations of LLP1
* Will be an in-memory test, which likely won't work for the full dataset.

The datasets we'll use:

In [1]:
did_LLP1 = "mc23_13p6TeV:mc23_13p6TeV.802746.Py8EG_Zprime2EJs_Ld20_rho40_pi10_Zp2600_l1.deriv.DAOD_LLP1.e8531_s4159_r15530_p6463"
did_PHYSLITE = "mc23_13p6TeV:mc23_13p6TeV.802746.Py8EG_Zprime2EJs_Ld20_rho40_pi10_Zp2600_l1.deriv.DAOD_PHYSLITE.e8531_s4159_r15530_p6491"

## Imports

In [2]:
import awkward as ak
import numpy as np
from func_adl_servicex_xaodr25 import FuncADLQueryPHYSLITE
from servicex import Sample, ServiceXSpec, dataset, deliver
from servicex_analysis_utils import to_awk

from calratio_training_data import RunConfig, fetch_training_data

## Fetching the data

First the PHYSLITE data. We have to do this by hand, of course.

In [3]:
# Define the base query
base_query = FuncADLQueryPHYSLITE()

# Query to fetch muons and MET
query = base_query.Select(
    lambda e: {
        "jets": e.Jets().Where(lambda j: j.pt() >= 40 and abs(j.eta()) < 2.5),
        "event_info": e.EventInfo("EventInfo"),
    }
).Select(
    lambda e: {
        "jet_pt": e.jets.Select(lambda jet: jet.pt() / 1000.0),
        "jet_eta": e.jets.Select(lambda jet: jet.eta()),
        "jet_phi": e.jets.Select(lambda jet: jet.phi()),
        "run": e.event_info.runNumber(),
        "event": e.event_info.eventNumber(),
    }
)

# Fetch the data
data = to_awk(
    deliver(
        ServiceXSpec(
            Sample=[
                Sample(
                    Name="did_PHYSLITE",
                    Dataset=dataset.Rucio(did_PHYSLITE),
                    Query=query,
                )
            ]
        ),
    )
)["did_PHYSLITE"]

# Next, reformat it so it is per-jet, the way our training data is
data_PHYSLITE = ak.values_astype(
    ak.zip(
        {
            "pt": ak.flatten(data["jet_pt"]),
            "eta": ak.flatten(data["jet_eta"]),
            "phi": ak.flatten(data["jet_phi"]),
            "runNumber": ak.flatten(
                ak.broadcast_arrays(data["run"], data["jet_pt"])[0], axis=1
            ),
            "eventNumber": ak.flatten(
                ak.broadcast_arrays(data["event"], data["jet_pt"])[0], axis=1
            ),
        },
        with_name="Momentum3D",
    ),
    np.float32,
)

data_PHYSLITE.type.show()

Output()

411807 * Momentum3D[
    pt: float32,
    eta: float32,
    phi: float32,
    runNumber: float32,
    eventNumber: float32
]


And the training data

* For this test we had to turn off the jet cleaning tool, as this version of LLP1 does not have the jet cleaning data. This was done by modifying the source code by hand (and hopefully not checking it in!).
    * Modify the `training_query.py` `good_training_jet` function - comment out the call to `jet_clean_llp`.

In [4]:
data_LLP1 = fetch_training_data(did_LLP1, RunConfig(run_locally=False, ignore_cache=False))
data_LLP1.type.show()

Output()

183797 * Momentum3D[
    runNumber: uint32,
    eventNumber: uint64,
    pt: float32,
    eta: float32,
    phi: float32,
    tracks: var * Momentum3D[
        eta: float32,
        phi: float32,
        pt: float32,
        vertex_nParticles: float32,
        d0: float32,
        z0: float32,
        chiSquared: float32,
        PixelShared: float32,
        SCTShared: float32,
        PixelHoles: float32,
        SCTHoles: float32,
        PixelHits: float32,
        SCTHits: float32
    ],
    clusters: var * Momentum3D[
        eta: float32,
        phi: float32,
        pt: float32,
        l1hcal: float32,
        l2hcal: float32,
        l3hcal: float32,
        l4hcal: float32,
        l1ecal: float32,
        l2ecal: float32,
        l3ecal: float32,
        l4ecal: float32,
        time: float32
    ],
    msegs: var * {
        etaPos: float32,
        phiPos: float32,
        etaDir: float32,
        phiDir: float32,
        t0: float32,
        chiSquared: float32
    }
]


## Combining

Now that we have `data_LLP1 and `data_PHYSLITE`, we need to combine the two event-by-event and jet-by-jet in the event.

In [6]:
# Let's examine the structure of both datasets
print("PHYSLITE data fields:")
print(data_PHYSLITE.fields)
print(f"PHYSLITE data length: {len(data_PHYSLITE)}")
print("\nLLP1 data fields:")
print(data_LLP1.fields)
print(f"LLP1 data length: {len(data_LLP1)}")

# Show a few examples
print("\nFirst few PHYSLITE jets:")
print(data_PHYSLITE[:3])
print("\nFirst few LLP1 jets:")
print(data_LLP1[:3])

PHYSLITE data fields:
['pt', 'eta', 'phi', 'runNumber', 'eventNumber']
PHYSLITE data length: 411807

LLP1 data fields:
['runNumber', 'eventNumber', 'pt', 'eta', 'phi', 'tracks', 'clusters', 'msegs']
LLP1 data length: 183797

First few PHYSLITE jets:
[{pt: 613, eta: 0.234, phi: -0.401, runNumber: 4.5e+05, ...}, {...}, {...}]

First few LLP1 jets:
[{runNumber: 450000, eventNumber: 38001, pt: 1.09e+03, eta: 0.0725, ...}, ...]


In [17]:
def match_jets_optimized(physlite_data, llp1_data, delta_r_threshold=0.4):
    """
    Optimized jet matching using vectorized operations where possible.
    
    Args:
        physlite_data: PHYSLITE jet data with calibrated pt, eta, phi
        llp1_data: LLP1 jet data with additional information (tracks, clusters, msegs)
        delta_r_threshold: Maximum delta R for jet matching
    
    Returns:
        Combined dataset with PHYSLITE kinematics and LLP1 additional information
    """
    
    print(f"Optimized matching between {len(physlite_data)} PHYSLITE jets and {len(llp1_data)} LLP1 jets...")
    
    # Convert to numpy for faster operations
    physlite_run = ak.to_numpy(physlite_data.runNumber).astype(np.int64)
    physlite_event = ak.to_numpy(physlite_data.eventNumber).astype(np.int64)
    physlite_eta = ak.to_numpy(physlite_data.eta).astype(np.float32)
    physlite_phi = ak.to_numpy(physlite_data.phi).astype(np.float32)
    
    llp1_run = ak.to_numpy(llp1_data.runNumber).astype(np.int64) 
    llp1_event = ak.to_numpy(llp1_data.eventNumber).astype(np.int64)
    llp1_eta = ak.to_numpy(llp1_data.eta).astype(np.float32)
    llp1_phi = ak.to_numpy(llp1_data.phi).astype(np.float32)
    
    # Create event keys
    physlite_event_keys = physlite_run * 1000000 + physlite_event
    llp1_event_keys = llp1_run * 1000000 + llp1_event
    
    print("Finding common events...")
    
    # Find common events efficiently
    common_events = np.intersect1d(physlite_event_keys, llp1_event_keys)
    print(f"Found {len(common_events)} common events")
    
    if len(common_events) == 0:
        print("No common events!")
        return None
    
    # Create masks for jets in common events
    physlite_mask = np.isin(physlite_event_keys, common_events)
    llp1_mask = np.isin(llp1_event_keys, common_events)
    
    print(f"PHYSLITE jets in common events: {np.sum(physlite_mask)}")
    print(f"LLP1 jets in common events: {np.sum(llp1_mask)}")
    
    # Extract jets in common events
    physlite_filtered_idx = np.where(physlite_mask)[0]
    llp1_filtered_idx = np.where(llp1_mask)[0]
    
    physlite_filtered_events = physlite_event_keys[physlite_mask]
    llp1_filtered_events = llp1_event_keys[llp1_mask]
    physlite_filtered_eta = physlite_eta[physlite_mask]
    physlite_filtered_phi = physlite_phi[physlite_mask]
    llp1_filtered_eta = llp1_eta[llp1_mask]
    llp1_filtered_phi = llp1_phi[llp1_mask]
    
    print("Matching jets within events...")
    
    # Group by event and match jets
    matched_physlite_idx = []
    matched_llp1_idx = []
    match_delta_r = []
    
    # Process each common event
    unique_events = np.unique(common_events)
    
    for i, event_key in enumerate(unique_events):
        if (i + 1) % 1000 == 0:
            print(f"Processed {i + 1}/{len(unique_events)} events")
            
        # Get jets for this event
        physlite_event_mask = physlite_filtered_events == event_key
        llp1_event_mask = llp1_filtered_events == event_key
        
        if not np.any(physlite_event_mask) or not np.any(llp1_event_mask):
            continue
            
        event_physlite_idx = physlite_filtered_idx[physlite_event_mask]
        event_llp1_idx = llp1_filtered_idx[llp1_event_mask]
        
        event_physlite_eta = physlite_filtered_eta[physlite_event_mask]
        event_physlite_phi = physlite_filtered_phi[physlite_event_mask]
        event_llp1_eta = llp1_filtered_eta[llp1_event_mask]
        event_llp1_phi = llp1_filtered_phi[llp1_event_mask]
        
        # Calculate distance matrix for this event
        delta_eta = event_physlite_eta[:, np.newaxis] - event_llp1_eta[np.newaxis, :]
        delta_phi = event_physlite_phi[:, np.newaxis] - event_llp1_phi[np.newaxis, :]
        
        # Handle phi wraparound
        delta_phi = np.where(np.abs(delta_phi) > np.pi,
                           delta_phi - 2*np.pi*np.sign(delta_phi),
                           delta_phi)
        
        # Calculate delta R
        delta_r = np.sqrt(delta_eta**2 + delta_phi**2)
        
        # Find best matches for each PHYSLITE jet
        min_delta_r = np.min(delta_r, axis=1)
        best_llp1_idx = np.argmin(delta_r, axis=1)
        
        # Apply threshold
        good_matches = min_delta_r < delta_r_threshold
        
        if np.any(good_matches):
            matched_physlite_idx.extend(event_physlite_idx[good_matches])
            matched_llp1_idx.extend(event_llp1_idx[best_llp1_idx[good_matches]])
            match_delta_r.extend(min_delta_r[good_matches])
    
    print(f"Found {len(matched_physlite_idx)} total matched jets")
    
    if len(matched_physlite_idx) == 0:
        print("No jets pass matching criteria!")
        return None
    
    # Create basic dataset first to avoid broadcasting issues
    print("Creating basic combined dataset...")
    combined_data = ak.zip({
        # Use calibrated kinematics from PHYSLITE
        "pt": physlite_data.pt[matched_physlite_idx],
        "eta": physlite_data.eta[matched_physlite_idx], 
        "phi": physlite_data.phi[matched_physlite_idx],
        
        # Use original (uncalibrated) kinematics from LLP1 for reference
        "pt_uncalib": llp1_data.pt[matched_llp1_idx],
        "eta_uncalib": llp1_data.eta[matched_llp1_idx],
        "phi_uncalib": llp1_data.phi[matched_llp1_idx],
        
        # Event information
        "runNumber": physlite_data.runNumber[matched_physlite_idx],
        "eventNumber": physlite_data.eventNumber[matched_physlite_idx],
        
        # Matching quality
        "delta_r_match": ak.Array(match_delta_r),
    })
    
    print("Adding nested LLP1 data...")
    # Add the nested fields separately to avoid broadcasting issues
    combined_data = ak.with_field(combined_data, llp1_data.tracks[matched_llp1_idx], "tracks")
    combined_data = ak.with_field(combined_data, llp1_data.clusters[matched_llp1_idx], "clusters")
    combined_data = ak.with_field(combined_data, llp1_data.msegs[matched_llp1_idx], "msegs")
    
    print(f"Combined dataset has {len(combined_data)} jets")
    print(f"Mean delta R: {ak.mean(combined_data.delta_r_match):.4f}")
    print(f"Max delta R: {ak.max(combined_data.delta_r_match):.4f}")
    
    # Show pt calibration factor distribution
    calib_factor = combined_data.pt / combined_data.pt_uncalib
    print(f"PT calibration factor - mean: {ak.mean(calib_factor):.3f}, std: {ak.std(calib_factor):.3f}")
    
    return combined_data

# Perform the optimized matching
print("Starting optimized jet matching...")
combined_jets = match_jets_optimized(data_PHYSLITE, data_LLP1)

Starting optimized jet matching...
Optimized matching between 411807 PHYSLITE jets and 183797 LLP1 jets...
Finding common events...
Found 30000 common events
PHYSLITE jets in common events: 411807
LLP1 jets in common events: 183797
Matching jets within events...
Processed 1000/30000 events
Processed 2000/30000 events
Processed 3000/30000 events
Processed 4000/30000 events
Processed 5000/30000 events
Processed 6000/30000 events
Processed 7000/30000 events
Processed 8000/30000 events
Processed 9000/30000 events
Processed 10000/30000 events
Processed 11000/30000 events
Processed 12000/30000 events
Processed 13000/30000 events
Processed 14000/30000 events
Processed 15000/30000 events
Processed 16000/30000 events
Processed 17000/30000 events
Processed 18000/30000 events
Processed 19000/30000 events
Processed 20000/30000 events
Processed 21000/30000 events
Processed 22000/30000 events
Processed 23000/30000 events
Processed 24000/30000 events
Processed 25000/30000 events
Processed 26000/30000

In [19]:
# Let's examine the results
print("=== JET MATCHING RESULTS ===")
print(f"Successfully matched {len(combined_jets):,} jets")
print(f"Available fields: {combined_jets.fields}")
print()

print("=== MATCHING QUALITY ===")
print(f"Delta R statistics:")
print(f"  Mean: {ak.mean(combined_jets.delta_r_match):.4f}")
print(f"  Median: {np.median(ak.to_numpy(combined_jets.delta_r_match)):.4f}") 
print(f"  Std: {ak.std(combined_jets.delta_r_match):.4f}")
print(f"  Max: {ak.max(combined_jets.delta_r_match):.4f}")
print()

print("=== CALIBRATION FACTORS ===")
pt_calib = combined_jets.pt / combined_jets.pt_uncalib
eta_diff = combined_jets.eta - combined_jets.eta_uncalib
phi_diff = combined_jets.phi - combined_jets.phi_uncalib

print(f"PT calibration (calibrated/uncalibrated):")
print(f"  Mean: {ak.mean(pt_calib):.4f}")
print(f"  Median: {np.median(ak.to_numpy(pt_calib)):.4f}")
print(f"  Std: {ak.std(pt_calib):.4f}")

print(f"Eta difference (calibrated - uncalibrated):")
print(f"  Mean: {ak.mean(eta_diff):.6f}")
print(f"  Std: {ak.std(eta_diff):.6f}")

print(f"Phi difference (calibrated - uncalibrated):")  
print(f"  Mean: {ak.mean(phi_diff):.6f}")
print(f"  Std: {ak.std(phi_diff):.6f}")
print()

print("=== ADDITIONAL DATA AVAILABILITY ===")
print(f"Tracks per jet - mean: {ak.mean(ak.num(combined_jets.tracks)):.1f}")
print(f"Clusters per jet - mean: {ak.mean(ak.num(combined_jets.clusters)):.1f}")
print(f"Msegs per jet - mean: {ak.mean(ak.num(combined_jets.msegs)):.1f}")
print()

print("=== EXAMPLE JETS ===")
for i in range(min(3, len(combined_jets))):
    print(f"Jet {i+1}:")
    print(f"  Event: Run {combined_jets.runNumber[i]}, Event {combined_jets.eventNumber[i]}")
    print(f"  PT: {combined_jets.pt_uncalib[i]:.1f} → {combined_jets.pt[i]:.1f} GeV (factor: {combined_jets.pt[i]/combined_jets.pt_uncalib[i]:.3f})")
    print(f"  Eta: {combined_jets.eta_uncalib[i]:.4f} → {combined_jets.eta[i]:.4f}")
    print(f"  Phi: {combined_jets.phi_uncalib[i]:.4f} → {combined_jets.phi[i]:.4f}")
    print(f"  Delta R: {combined_jets.delta_r_match[i]:.4f}")
    print(f"  Tracks: {len(combined_jets.tracks[i])}, Clusters: {len(combined_jets.clusters[i])}, Msegs: {len(combined_jets.msegs[i])}")
    print()

print("SUCCESS: Jet matching complete! 🎉")
print("You now have a combined dataset with:")
print("- Calibrated pt, eta, phi from PHYSLITE")
print("- Tracks, clusters, and msegs from LLP1") 
print("- All properly matched by event and proximity in eta-phi space")

=== JET MATCHING RESULTS ===
Successfully matched 188,311 jets
Available fields: ['pt', 'eta', 'phi', 'pt_uncalib', 'eta_uncalib', 'phi_uncalib', 'runNumber', 'eventNumber', 'delta_r_match', 'tracks', 'clusters', 'msegs']

=== MATCHING QUALITY ===
Delta R statistics:
  Mean: 0.0456
  Median: 0.0196
  Std: 0.0705
  Max: 0.4000

=== CALIBRATION FACTORS ===
PT calibration (calibrated/uncalibrated):
  Mean: 0.9595
  Median: 0.9968
  Std: 0.2235
Eta difference (calibrated - uncalibrated):
  Mean: -0.000003
  Std: 0.057810
Phi difference (calibrated - uncalibrated):
  Mean: 0.000401
  Std: 0.421882

=== ADDITIONAL DATA AVAILABILITY ===
Tracks per jet - mean: 2.6
Clusters per jet - mean: 0.0
Msegs per jet - mean: 5.4

=== EXAMPLE JETS ===
Jet 1:
  Event: Run 450000.0, Event 30001.0
  PT: 267.6 → 298.9 GeV (factor: 1.117)
  Eta: -1.2392 → -1.2348
  Phi: -0.4424 → -0.4539
  Delta R: 0.0124
  Tracks: 10, Clusters: 0, Msegs: 1

Jet 2:
  Event: Run 450000.0, Event 30001.0
  PT: 200.9 → 204.2 GeV (

In [9]:
# Let's debug why no matches were found
print("PHYSLITE run/event ranges:")
print(f"Run range: {ak.min(data_PHYSLITE.runNumber)} - {ak.max(data_PHYSLITE.runNumber)}")
print(f"Event range: {ak.min(data_PHYSLITE.eventNumber)} - {ak.max(data_PHYSLITE.eventNumber)}")

print("\nLLP1 run/event ranges:")
print(f"Run range: {ak.min(data_LLP1.runNumber)} - {ak.max(data_LLP1.runNumber)}")
print(f"Event range: {ak.min(data_LLP1.eventNumber)} - {ak.max(data_LLP1.eventNumber)}")

# Check for any overlapping runs using numpy for unique
physlite_runs = np.unique(ak.to_numpy(data_PHYSLITE.runNumber))
llp1_runs = np.unique(ak.to_numpy(data_LLP1.runNumber))
common_runs = [r for r in physlite_runs if r in llp1_runs]
print(f"\nCommon runs: {common_runs}")
print(f"PHYSLITE runs: {physlite_runs}")
print(f"LLP1 runs: {llp1_runs}")

# Let's also check some specific events
print(f"\nFirst 10 PHYSLITE events (run, event):")
for i in range(min(10, len(data_PHYSLITE))):
    print(f"  ({data_PHYSLITE.runNumber[i]}, {data_PHYSLITE.eventNumber[i]})")

print(f"\nFirst 10 LLP1 events (run, event):")  
for i in range(min(10, len(data_LLP1))):
    print(f"  ({data_LLP1.runNumber[i]}, {data_LLP1.eventNumber[i]})")

# Check if there are any events in common at all
physlite_events = set()
for i in range(len(data_PHYSLITE)):
    physlite_events.add((float(data_PHYSLITE.runNumber[i]), float(data_PHYSLITE.eventNumber[i])))

llp1_events = set()
for i in range(len(data_LLP1)):
    llp1_events.add((float(data_LLP1.runNumber[i]), float(data_LLP1.eventNumber[i])))

common_events = physlite_events.intersection(llp1_events)
print(f"\nNumber of common events: {len(common_events)}")
if len(common_events) > 0:
    print(f"First few common events: {list(common_events)[:5]}")

PHYSLITE run/event ranges:
Run range: 450000.0 - 450000.0
Event range: 30001.0 - 60000.0

LLP1 run/event ranges:
Run range: 450000 - 450000
Event range: 30001 - 60000

Common runs: [np.float32(450000.0)]
PHYSLITE runs: [450000.]
LLP1 runs: [450000]

First 10 PHYSLITE events (run, event):
  (450000.0, 32060.0)
  (450000.0, 32060.0)
  (450000.0, 32060.0)
  (450000.0, 32060.0)
  (450000.0, 32060.0)
  (450000.0, 32060.0)
  (450000.0, 32060.0)
  (450000.0, 32060.0)
  (450000.0, 32060.0)
  (450000.0, 32060.0)

First 10 LLP1 events (run, event):
  (450000, 38001)
  (450000, 38001)
  (450000, 38001)
  (450000, 38001)
  (450000, 38001)
  (450000, 38001)
  (450000, 38001)
  (450000, 38009)
  (450000, 38009)
  (450000, 38009)

Number of common events: 30000
First few common events: [(450000.0, 41535.0), (450000.0, 35119.0), (450000.0, 57750.0), (450000.0, 51334.0), (450000.0, 44918.0)]


## Combining

This combination is done in memory to test out the basic mechanism.

In [5]:
def group_by_event(d):
    key = ak.zip({"run": d.runNumber, "evt":d.eventNumber}, depth_limit=1)
    run_ordered = ak.argsort(key.run, stable=True)
    run_runs = ak.run_lengths(key[run_ordered].run)
    key_by_event = ak.unflatten(key[run_ordered], run_runs)

    event_ordered = ak.argsort(key_by_event.evt, stable=True, axis=-1)
    event_runs = ak.run_lengths(key_by_event[event_ordered].evt)
    group_event = ak.unflatten(key_by_event[event_ordered], ak.flatten(event_runs), axis=-1)

    return group_event

group_event_PHYSLITE = group_by_event(data_PHYSLITE)
group_event_LLP1 = group_by_event(data_LLP1)

In [71]:
group_event_LLP1[:].run[0][0]

<Array [450000, 450000, 450000, ..., 450000, 450000, 450000] type='9 * uint32'>

In [56]:
ak.max(group_event_LLP1[0].evt, keepdims=True, axis=1)

<Array [[30001], [30002], ..., [59999], [60000]] type='30000 * 1 * ?uint64'>